In [ ]:
import os
import pandas as pd
import pprint
import numpy as np
import zipfile
import shutil
import matplotlib.pyplot as plt
import csv

In [ ]:
imu_data = ["Imu0", "Imu1", "Imu2"]
imu_attributes = ["linear_accleration_x", "linear_accleration_y", "linear_accleration_z", "angular_velocity_x", "angular_velocity_y", "angular_velocity_z",                   "orientation_x", "orientation_y", "orientation_z", "orientation_w"]
imu_transformations = ["imu0_to_imu1", "imu0_to_imu2"]

COLUMNS = [f"{imu}_{attr}" for imu in imu_data for attr in imu_attributes]
COLUMNS += [f"{transformation}_translation_{axis}" for transformation in imu_transformations for axis in ["x", "y", "z"]]
COLUMNS += [f"{transformation}_rotation_{axis}" for transformation in imu_transformations for axis in ["x", "y", "z", "w"]]

# Load Data

In [ ]:
GESTURES = [
    "STATIC",
    "SLIDE_UP",
    "SLIDE_DOWN",
    "SLIDE_LEFT",
    "SLIDE_RIGHT",
    "RELEASE",
    "GRASP",
    "HIGHLIGHT",
    "ON_YES",
    "OFF_NO",
    "NONE",
]

def load_data(filepath='/home/ubuntu/FYP-ROS/rosbag/bag/info.txt'):
    dataset = {}
    with open(filepath, 'r') as f:
        for line in f:
            line = line.strip()
            for gesture in GESTURES:
                if line.startswith(f'{gesture}:'):
                    key = gesture
                    dataset[key] = []
                    break
            else:
                if line != '':
                    dataset[key].append(line.split()[-1])
    return dataset

In [ ]:
dataset = load_data()
for key in dataset:
    print(f'{key}: {len(dataset[key])}')

# Data Augmentation Algo

In [ ]:
# Adding normal distributed noise
def jitter(x, sigma=0.03):
  return x + np.random.normal(loc=0., scale=sigma, size=x.shape)

# Scaling (Quite literally)
def scaling(x, sigma=0.05):
  factor = np.random.normal(loc=1., scale=sigma, size=x.shape)
  return np.multiply(x, factor)

def time_warp(x, sigma=0.1, knot=4):
  from scipy.interpolate import CubicSpline
  orig_steps = np.arange(x.shape[0])

  random_warps = np.random.normal(loc=1.0, scale=sigma, size=(knot+2))
  warp_steps = np.linspace(0, x.shape[0]-1., num=knot+2).T

  time_warp = CubicSpline(warp_steps, warp_steps * random_warps)(orig_steps)
  scale = (x.shape[0]-1)/time_warp[-1]

  x = x.T
  ret = np.zeros_like(x)

  for col_index, col in enumerate(x):
      ret[col_index] = np.interp(orig_steps, np.clip(
          scale * time_warp, 0, x.shape[1]-1), col)
  return ret.T


# Process

In [ ]:
data_root_dir = "/home/ubuntu/FYP-ROS/rosbag/data"

for gesture_type in dataset:
  for data_id in dataset[gesture_type]:
    original_data_df = pd.read_csv(f"{data_root_dir}/data_clean/{data_id}_data.csv")
    original_label_df = pd.read_csv(f"{data_root_dir}/label_clean/{data_id}_label.csv")
    
    timestamp_column = original_data_df.pop("timestamp")
    numpy_df = original_data_df.to_numpy()[np.newaxis, :]

    jitter_df = pd.DataFrame(jitter(numpy_df[0], sigma=0.05), columns=COLUMNS)
    # scaling_df = pd.DataFrame(scaling(numpy_df[0]), columns=COLUMNS)
    time_warp_df  = pd.DataFrame(time_warp(numpy_df[0], sigma=0.1), columns=COLUMNS)
    time_warp_df2 = pd.DataFrame(time_warp(numpy_df[0], sigma=0.125), columns=COLUMNS)

    # # Add timestamp column back
    original_data_df = pd.concat([timestamp_column, original_data_df], axis=1)
    jitter_df = pd.concat([timestamp_column, jitter_df], axis=1)
    # scaling_df = pd.concat([timestamp_column, scaling_df], axis=1)
    time_warp_df  = pd.concat([timestamp_column, time_warp_df], axis=1)
    time_warp_df2 = pd.concat([timestamp_column, time_warp_df2], axis=1)

    # # Write to new data csv
    original_data_df.to_csv(f"{data_root_dir}/data_clean_augment/{data_id}_data.csv", index=False)
    jitter_df.to_csv(f"{data_root_dir}/data_clean_augment/{data_id}_data_jitter.csv", index=False)
    # scaling_df.to_csv(f"{data_root_dir}/data_clean_augment/{data_id}_data_scaling.csv", index=False)
    time_warp_df.to_csv(f"{data_root_dir}/data_clean_augment/{data_id}_data_time_warp.csv", index=False)
    time_warp_df2.to_csv(f"{data_root_dir}/data_clean_augment/{data_id}_data_time_warp2.csv", index=False)

    # # Write to new label csv
    original_label_df.to_csv(f"{data_root_dir}/label_clean_augment/{data_id}_label.csv", index=False)
    original_label_df.to_csv(f"{data_root_dir}/label_clean_augment/{data_id}_label_jitter.csv", index=False)
    # original_label_df.to_csv(f"{data_root_dir}/label_clean_augment/{data_id}_label_scaling.csv", index=False)
    original_label_df.to_csv(f"{data_root_dir}/label_clean_augment/{data_id}_label_time_warp.csv", index=False)
    original_label_df.to_csv(f"{data_root_dir}/label_clean_augment/{data_id}_label_time_warp2.csv", index=False)

# Plot

In [ ]:
fig, axs = plt.subplots(3, 6, figsize=(20, 10))

data_id = np.random.choice(dataset['STATIC'])

raw_data = pd.read_csv(f"{data_root_dir}/data_clean_augment/{data_id}_data.csv")
augment_data = pd.read_csv(f"{data_root_dir}/data_clean_augment/{data_id}_data_jitter.csv")

acc_axes = axs[:, :3].ravel()
vel_axes = axs[:, 3:].ravel()
acc_data = [raw_data[f'Imu{i}_linear_accleration_{xyz}'] for i in range(3) for xyz in ['x', 'y', 'z']]
augment_acc_data = [augment_data[f'Imu{i}_linear_accleration_{xyz}'] for i in range(3) for xyz in ['x', 'y', 'z']]
vel_data = [raw_data[f'Imu{i}_angular_velocity_{xyz}'] for i in range(3) for xyz in ['x', 'y', 'z']]
augment_vel_data = [augment_data[f'Imu{i}_angular_velocity_{xyz}'] for i in range(3) for xyz in ['x', 'y', 'z']]
acc_titles = [f'Imu{i}_acc_{xyz}' for i in range(3) for xyz in ['x', 'y', 'z']]
vel_titles = [f'Imu{i}_vel_{xyz}' for i in range(3) for xyz in ['x', 'y', 'z']]

for ax, data, augment_ax, title in zip(acc_axes, acc_data, augment_acc_data, acc_titles):
    ax.plot(data, label="raw", color="red")
    ax.plot(augment_ax, label="augment", color="blue")
    ax.set_title(title)
    ax.set_ylim([-1.5 * 9.8, 1.5 * 9.8])

for ax, data,augment_ax, title in zip(vel_axes, vel_data, augment_vel_data, vel_titles):
    ax.plot(data, label="raw", color="red")
    ax.plot(augment_ax, label="augment", color="blue")
    ax.set_title(title)
    ax.set_ylim([-5, 5])

fig.legend(['raw', 'augment'])
fig.suptitle(f"Data Augmentation for {data_id}")
plt.show()
